In [25]:
import os
import nengo
import nengo_dl
import numpy as np
import keras
import tensorflow as tf
import sklearn.model_selection
from tensorflow.python.keras import Model, Sequential, Input
from tensorflow.python.keras.layers import Conv2D, BatchNormalization, Dropout, AveragePooling2D, Flatten, Dense


In [26]:
dataset = np.load(file=os.path.join('..', 'datasets', 'output', 'p300-target-nontarget.npz'))
target_x, non_target_x = dataset['target_features'], dataset['non_target_features']

# trim target or non target data to have same amount of samples
if target_x.shape[0] > non_target_x.shape[0]:
    target_x = target_x[:non_target_x.shape[0]]
else:
    non_target_x = non_target_x[:target_x.shape[0]]

print('target_x:', target_x.shape, 'non_target_x:', non_target_x.shape)

# (1, 0) vector for target data
target_y = np.tile(np.array([1, 0]), (target_x.shape[0], 1))
# (0, 1) vector for non target data
non_target_y = np.tile(np.array([0, 1]), (non_target_x.shape[0], 1))
print('target_y:', target_y, 'non_target_y:', non_target_y)

features = np.concatenate((target_x, non_target_x), axis=0)
labels = np.vstack((target_y, non_target_y))
features = np.expand_dims(features, -1)
print(features, labels)

target_x: (1525, 3, 1201) non_target_x: (1525, 3, 1201)
target_y: [[1 0]
 [1 0]
 [1 0]
 ...
 [1 0]
 [1 0]
 [1 0]] non_target_y: [[0 1]
 [0 1]
 [0 1]
 ...
 [0 1]
 [0 1]
 [0 1]]
[[[[-8.04011680e-06]
   [-8.70710899e-06]
   [-8.82429649e-06]
   ...
   [-4.60801559e-05]
   [-4.59580855e-05]
   [-4.65410934e-05]]

  [[-1.91131598e-05]
   [-1.98250738e-05]
   [-1.99373785e-05]
   ...
   [-5.13797613e-05]
   [-5.14608160e-05]
   [-5.21395270e-05]]

  [[-1.35353725e-05]
   [-1.47272670e-05]
   [-1.53405482e-05]
   ...
   [-4.72189662e-05]
   [-4.79357631e-05]
   [-4.91300990e-05]]]


 [[[ 7.30717020e-06]
   [ 8.66459208e-06]
   [ 9.64115458e-06]
   ...
   [-3.21293532e-05]
   [-3.20863845e-05]
   [-3.22665603e-05]]

  [[ 6.92052433e-07]
   [ 2.11197431e-06]
   [ 3.23892743e-06]
   ...
   [-3.16458382e-05]
   [-3.16995491e-05]
   [-3.20110726e-05]]

  [[ 8.42213250e-06]
   [ 9.38160516e-06]
   [ 9.87623406e-06]
   ...
   [-5.55348987e-05]
   [-5.55178089e-05]
   [-5.56154652e-05]]]


 [[[-3.077

In [27]:
# global seed for consistency
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

In [28]:
def create_nn():
    input_layer = Input(shape=(3, 1201, 1))
    conv_input = Conv2D(filters=6, kernel_size=(3, 3), activation=tf.nn.elu)(input_layer)
    batch_norm1 = BatchNormalization()(conv_input)
    dropout1 = Dropout(0.5)(batch_norm1)
    avg_pooling = AveragePooling2D(pool_size=(1, 8))(dropout1)
    flatten = Flatten()(avg_pooling)
    dense1 = Dense(100, activation=keras.activations.elu)(flatten)
    batch_norm2 = BatchNormalization()(dense1)
    dropout2 = Dropout(0.5)(batch_norm2)
    output_layer = Dense(2, activation=keras.activations.softmax)(dropout2)

    return Model(inputs=input_layer, outputs=output_layer), input_layer, output_layer


In [29]:
def run_spiking(
        model, input_layer, output_layer, activation, test_data_x, test_data_y, params_file='keras_to_snn_params',
        scale_firing_rates=1, synapse=None
):
    converter = nengo_dl.Converter(model, swap_activations={ tf.nn.elu: activation },
                                   scale_firing_rates=scale_firing_rates, synapse=synapse)

    nengo_input, nengo_output = converter.inputs[input_layer], converter.inputs[output_layer]

    with nengo_dl.Simulator(converter.net, minibatch_size=10, progress_bar=False) as nengo_sim:
        nengo_sim.load_params(params_file)
        data = nengo_sim.predict({ nengo_input: test_data_x })

    predictions = np.argmax(data[nengo_output][:, -1], axis=-1)
    acc = (predictions == test_data_y[:, 0, 0]).mean()
    print('accuracy: {:4f}%'.format(acc*100))
    return acc


def run_model(
        validation_metrics, test_metrics, snn_metrics,
        train_x, train_y, validation_x, validation_y, test_x, test_y
):
    model, input_layer, output_layer = create_nn()

    print('rank:', len(test_y.shape))

    early_stop_callback = keras.callbacks.EarlyStopping(patience=5, verbose=1, restore_best_weights=True)

    converter = nengo_dl.Converter(model)

    with nengo_dl.Simulator(converter.net, minibatch_size=10) as sim:
        sim.compile(
            optimizer=tf.optimizers.Adam(),
            loss=tf.losses.BinaryCrossentropy(),
            metrics=[tf.metrics.binary_accuracy]
        )

        nengo_input, nengo_output = converter.inputs[input_layer], converter.outputs[output_layer]

        hist = sim.fit(
            { nengo_input: train_x },
            { nengo_output: train_y },
            callbacks=[early_stop_callback],
            epochs=30,
            batch_size=16,
            shuffle=True,
            validation_data=(
                { nengo_input: validation_x },
                { nengo_output: validation_y }
            )
        )
        validation_metrics.append(hist)

        acc, loss = sim.evaluate(test_x, test_y)
        test_metrics.append((acc, loss))

        sim.save_params('./keras_to_snn_params')
        snn_acc = run_spiking(
            model=model,
            input_layer=input_layer,
            output_layer=output_layer,
            test_data_x=test_x, test_data_y=test_y,
            scale_firing_rates=20,
            activation=nengo.SpikingRectifiedLinear(),
            synapse=.01
        )
        snn_metrics.append(snn_acc)


In [30]:
# Monte Carlo Cross Validation

# split data into 3/4 as training and 1/4 as testing
train_x, test_x, train_y, test_y = sklearn.model_selection.train_test_split(
    features, labels, test_size=0.25, random_state=seed
)

shuffle_split = sklearn.model_selection.ShuffleSplit(
    n_splits=30, test_size=0.25, random_state=seed
)

iter = 0
validation_metrics, test_metrics, snn_metrics = [], [], []
for training, validation in shuffle_split.split(train_x, train_y):
    run_model(
        validation_metrics=validation_metrics,
        test_metrics=test_metrics,
        snn_metrics=snn_metrics,
        train_x = train_x[training],
        train_y= train_y[training],
        validation_x= train_x[validation],
        validation_y= train_y[validation],
        test_x=test_x,
        test_y=test_y
    )
    iter += 1

print(validation_metrics, test_metrics, snn_metrics)

rank: 2
Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               


C:\Users\itznu\anaconda3\envs\datascience-gpu\lib\site-packages\nengo_dl\converter.py:563: UserWarning: Activation type <function elu at 0x000001926233D0D8> does not have a native Nengo equivalent; falling back to a TensorNode
  "falling back to a TensorNode" % activation
C:\Users\itznu\anaconda3\envs\datascience-gpu\lib\site-packages\nengo_dl\converter.py:309: UserWarning: Cannot convert BatchNormalization layer to native Nengo objects unless inference_only=True. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
C:\Users\itznu\anaconda3\envs\datascience-gpu\lib\site-packages\nengo_dl\converter.py:309: UserWarning: Layer type <class 'tensorflow.python.keras.layers.core.Dropout'> does not have a registered converter. Falling back to TensorNode.
  % (error_msg + ". " if error_msg else "")
C:\Users\itznu\anaconda3\envs\datascience-gpu\lib\site-packages\nengo_dl\converter.py:563: UserWarning: Activation type <function elu at 0x0000019264D00828> does not have a native 

ValidationError: input_4 data: should have rank 3 (batch_size, n_steps, dimensions), found rank 4